<a href="https://colab.research.google.com/github/moey920/NLP/blob/master/%EC%98%81%EC%88%98%EC%A6%9D_%EB%A7%9E%EC%B6%A4%EB%B2%95_seq2seq%EB%A1%9C_%ED%95%B4%EA%B2%B0%ED%95%98%EA%B8%B0(receipt_seq2seq)%EC%9D%98_%EC%88%98%EC%A0%95%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

본 문서는 케라스를 이용해 RNN(Recurrent Neural Networks)모델인 Seq2Seq를 10분 안에 알려주는 튜토리얼 한글 버전입니다. Seq2Seq의 의미부터 케라스를 이용한 모델 구현을 다루고 있으며 본 문서 대상자는 recurrent networks와 keras에 대한 경험이 있다는 가정하에 진행합니다.

Keras
RNN
LSTM
NLP
Seq2Seq
GRU layer

원문 : A ten-minute introduction to sequence-to-sequence learning in Keras

# sequence-to-sequence 학습이란?

sequence-to-sequence(Seq2Seq) 학습은 한 도메인(예: 영어 문장)에서 다른 도메인(예: 불어로 된 문장)으로 시퀀스(sequence)를 변환하는 모델 학습을 의미합니다.

-  "the cat sat on the mat" -> [Seq2Seq model] -> "le chat etait assis sur le tapis"

이 모델은 기계 번역 혹은 자유로운 질의응답에 사용됩니다. (자연어 질문을 주어 자연어 응답을 생성) –일반적으로, 텍스트를 생성해야 할 경우라면 언제든지 적용할 수 있습니다.

해당 작업을 다루는 여러 가지 방법이(RNN 혹은 1D convnets) 있습니다.

## 자명한(명확한) 사례 : 입력과 출력 시퀀스 길이가 같을 때

입력과 출력 시퀀스 길이가 같을 경우, 케라스 Long Short-Term Memory(LSTM)이나 GRU 계층(혹은 다수의 계층) 같은 모델들을 간단하게 구현할 수 있습니다. 예제 스크립트에선 어떻게 RNN으로 문자열로 인코딩된 숫자들에 대한 덧셈 연산을 학습할 수 있는지 보여주고 있습니다.

이 방법의 주의점은 주어진 input[...t]으로 target[...t]을 생성 가능하다고 가정하는 것입니다. 일부 경우(예: 숫자된 문자열 추가)에선 정상적으로 작동하지만, 대부분의 경우에는 작동하지 않습니다. 일반적으론, 목표 시퀀스를 생성하기 위해 전체 입력 시퀀스 정보가 필요합니다.

## 일반 사례 : 표준 sequence-to-sequence

일반적으론 입력과 출력 시퀀스 길이가 다르고(예: 기계 번역) 목표 시퀀스를 예측하기 위해 전체 입력 시퀀스 정보가 필요합니다. 이를 위해 고급 설정이 필요하며, 일반적으로 “Seq2Seq models”를 언급할 때 참조합니다. 동작 방법은 하단을 참조하시면 되겠습니다.

- 하나(혹은 여러 개)의 RNN 계층은 “encoder” 역할을 합니다 : 입력 시퀀스를 처리하고 자체 내부 상태를 반환합니다. 여기서, encoder RNN의 결과는 사용하지 않고 상태만 복구시킵니다. 이 상태가 다음 단계에서 decoder의 “문맥” 혹은 “조건” 역할을 합니다.
- 또 하나(혹은 여러 개)의 RNN 계층은 “decoder” 역할을 합니다 : 목표 시퀀스에서 이전 문자들에 따라 다음 문자들을 예측하도록 훈련됩니다. 상세히 말하면, 목표 시퀀스를 같은 시퀀스로 바꾸지만 후에 “teacher forcing”이라는 학습 과정인, 한 개의 time step만큼 offset*이 되도록 훈련됩니다. 중요한 건, encoder는 encoder 상태 벡터들을 초기 상태로 사용하고 이는 decoder가 생성할 정보를 얻는 방법이기도 합니다. 사실, decoder는 주어진 target[...t]을 입력 시퀀스에 맞춰서 target[t+1...]을 생성하는 법을 학습합니다.
- offset 의 예: 문자 A의 배열이 ‘abcdef’를 가질 때, ‘c’가 A 시작점에서 2의 offset을 지님

추론 방식(즉: 알 수 없는 입력 시퀀스를 해석하려고 할 때)에선 약간 다른 처리를 거치게 됩니다.

- 1) 입력 시퀀스를 상태 벡터들로 바꿉니다.
- 2) 크기가 1인 목표 시퀀스로 시작합니다. (시퀀스의 시작 문자에만 해당)
- 3) 상태 벡터들과 크기가 1인 목표 시퀀스를 decoder에 넣어 다음 문자에 대한 예측치를 생성합니다.
- 4) 이런 예측치들을 사용해 다음 문자의 표본을 뽑습니다.(간단하게 argmax를 사용)
- 5) 목표 시퀀스에 샘플링된 문자를 붙입니다.
- 6) 시퀀스 종료 문자를 생성하거나 끝 문자에 도달할 때까지 앞의 과정을 반복합니다.

이같은 과정은 “teacher forcing” 없이 Seq2Seq를 학습시킬 때 쓰일 수도 있습니다. (decoder의 예측치들을 decoder에 다시 기재함으로써)

## 케라스 예제

실제 코드를 통해 위의 아이디어들을 설명하겠습니다.

예제를 구현하기 위해, 영어 문장과 이에 대해 불어로 번역한 문장 한 쌍으로 구성된 데이터 세트를 사용합니다. (manythings.org/anki에서 내려받을 수 있습니다.) 다운받을 파일은 fra-eng.zip입니다. 입력 문자를 문자 단위로 처리하고, 문자 단위로 출력문자를 생성하는 문자 수준 Seq2Seq model을 구현할 예정입니다. 또 다른 옵션은 기계 번역에서 좀 더 일반적인 단어 수준 model입니다. 글 끝단에서, Embedding계층을 사용하여 설명에 쓰인 model을 단어 수준 model로 바꿀 수 있는 참고 사항도 보실 수 있습니다.

설명에 쓰인 예제 전체 code는 Github에서 보실 수 있습니다.

진행 과정 요약으론:

- 1) 문장들을 3차원 배열(encoder_input_data, decoder_input_data, decoder_target_data)로 변환합니다.
encoder_input_data는 (num_pairs, max_english_sentence_length, num_english_characters) 형태의 3차원 배열로 영어 문장의 one-hot 형식 벡터 데이터를 갖고 있습니다.
decoder_input_data는 (num_pairs, max_french_sentence_length, num_french_characters)형태의 3차원 배열로 불어 문장의 one-hot형식 벡터 데이터를 갖고 있습니다.
decoder_target_data는 decoder_input_data와 같지만 하나의 time step만큼 offset 됩니다. decoder_target_data[:, t, :]는 decoder_input_data[:, t + 1, :]와 같습니다.
- 2) 기본 LSTM 기반의 Seq2Seq model을 주어진 encoder_input_data와 decoder_input_data로 decoder_target_data를 예측합니다. 해당 model은 teacher forcing을 사용합니다.
- 3) model이 작동하는지 확인하기 위해 일부 문장을 디코딩(decoding)합니다. (encoder_input_data의 샘플을 decoder_target_data의 표본으로 변환합니다.)
(문장을 디코딩하는)학습 단계와 추론 단계는 꽤나 다르기 때문에, 같은 내부 계층을 사용하지만 서로 다른 모델을 사용합니다.

다음은 원문 저자가 제공하는 model로 keras RNN의 3가지 핵심 특징들을 사용합니다:

- return_state는 encoder의 출력과 내부 RNN 상태인 리스트를 반환하도록 RNN을 구성하는 인수입니다. 이는 encoder의 상태를 복구하는 데 사용합니다.
- inital_state는 RNN의 초기 상태를 지정하는 인수입니다. 초기 상태로 incoder를 decoder로 전달하는 데 사용합니다.
- return_sequences는 출력된 전체 시퀀스를 반환하도록 구성하는 인수(마지막 출력을 제외하곤 기본 동작)로 decoder에 사용합니다.

예시용 코드입니다. 실행은 아래로 쭉 내려가서 진행해주세요.

In [0]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# 입력 시퀀스의 정의와 처리
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# `encoder_outputs`는 버리고 상태(`state_h, state_c`)는 유지
encoder_states = [state_h, state_c]

# `encoder_states`를 초기 상태로 사용해 decoder를 설정
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# 전체 출력 시퀀스를 반환하고 내부 상태도 반환하도록 decoder를 설정. 
# 학습 모델에서 상태를 반환하도록 하진 않지만, inference에서 사용할 예정.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
   
# `encoder_input_data`와 `decoder_input_data`를 `decoder_target_data`로 반환하도록 모델을 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

밑의 2줄로 샘플의 20%를 검증 데이터 세트로 손실을 관찰하면서 모델을 학습시킵니다.

In [0]:
# 학습 실행
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

맥북 CPU에서 1시간 정도 학습한 후에, 추론할 준비가 됩니다. 테스트 문장을 decode하기 위해 반복 수행할 것입니다.

- 1) 입력문장을 encode하고 초기 상태에 decoder의 상태를 가지고 옵니다.
- 2) 초기 상태 decoder의 한 단계와 “시퀀스 시작” 토큰을 목표로 실행합니다. 출력은 다음 목표 문자입니다.
- 3) 예측된 목표 문자를 붙이고 이를 반복합니다.
다음은 추론을 설정한 부분입니다.

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

아래의 코드는 위의 추론 루프를 구현하는 데 사용했습니다.

In [0]:
def decode_sequence(input_seq):
    # 상태 벡터로서 입력값을 encode
    states_value = encoder_model.predict(input_seq)

    # 길이가 1인 빈 목표 시퀀스를 생성
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 대상 시퀀스 첫 번째 문자를 시작 문자로 기재.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 시퀀스들의 batch에 대한 샘플링 반복(간소화를 위해, 배치 크기는 1로 상정)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 토큰으로 샘플링
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 탈출 조건 : 최대 길이에 도달하거나
        # 종료 문자를 찾을 경우
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # (길이 1인) 목표 시퀀스 최신화
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태 최신화
        states_value = [h, c]

    return decoded_sentence

몇 가지 좋은 결과를 얻게 됩니다. (학습 테스트에서 추출한 샘플을 해독하기에 놀랄만한 결과는 아니지만..)

Input sentence: Be nice.
Decoded sentence: Soyez gentil !
-

Input sentence: Drop it!
Decoded sentence: Laissez tomber !
-

Input sentence: Get out!
Decoded sentence: Sortez !

# 영수증 Seq2Seq

In [0]:
 
'''
#Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
**Summary of the algorithm**
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
**Data download**
[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)
[Lots of neat sentence pairs datasets.
](http://www.manythings.org/anki/)
**References**
- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

# epochs와 latent_dim의 초기값은 100, 256입니다. 하지만 낮을수록 훈련에 실패할 확률이 있으니 높게 설정해보세요
batch_size = 64  # Batch size for training. 한번에 훈련시킬 배치사이즈 설정
epochs = 40 # Number of epochs to train for. # 초기 한-영 번역기 훈련시 에포크 1회에 5분, 총 500분 이상 소요되었습니다. 코랩에서 GPU설정을 해주세요.
#또한 epochs 45번째 이상에서 약간의 과대적합이 일어나고 더이상 성능이 향상되지 않았습니다. / 2020-04-28 latent_dim을 1024로 설정했을 때 epochs 36 이상이면 과대적합 발생
# GPU를 사용해주세요. 
latent_dim = 1024  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on. 학습데이터 개수
# Path to the data txt file on disk.
data_path = '/content/drive/My Drive/캐시카우_노하람인턴_공유폴더/seq2seq/pre-post_0429.txt' #제작한 데이터 경로를 설정해주세요, 텍스트 파일이 UTF-8로 인코딩 되어있지 않으면 오류가 발생합니다.


# Vectorize the data.
input_texts = [] # 빈 리스트 생성
target_texts = [] # 빈 리스트 생성
input_characters = set() # 빈 집합 생성
target_characters = set() # 빈 집합 생성
# 중복을 허용하지 않는 set의 특징은 자료형의 중복을 제거하기 위한 필터 역할로 종종 사용하기도 한다.


with open(data_path, 'r', encoding='utf-8') as f:       # data_path를 읽기모드로, utf-8로 인코딩하여 f라는 이름으로 로드한다
    lines = f.read().split('\n')                        # f를 \n을 기준으로 나누어 한 줄씩 읽어 lines 변수에 저장한다.
for line in lines[: min(num_samples, len(lines) - 1)]:  # line을 lines까지, 처음부터 lines의 개수
                                                        # 훈련 샘플의 개수(10000)개와 실제 로드한 데이터의 개수(len(lines)-1)을 비교하여 최고값을 리턴하고, 처음부터 최소값까지 반복한다.
                                                        # 내장함수 min() : 반복가능한 객체의 가장 작은 요소 값을 리턴
    input_text, target_text, _ = line.split('\t')       # input_text(인풋), target_text(아웃풋), _(인덱스)는 한 문장으로 구성되어 있어 각각의 요소를 \t(TAB)으로 구분한다.
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.    "\t"문자를 시작 문자로, "\n"문자를 종료 문자로 사용.
    target_text = '\t' + target_text + '\n'             # targer_text는 앞에 '\t', 뒤에 '\n'을 붙인다.
    input_texts.append(input_text)                      # input_texts 라는 빈 리스트에 input_text를 하나씩 집어넣는다. 
    target_texts.append(target_text)                    # target_texts 라는 빈 리스트에 target_text를 하나씩 집어넣는다. 
    for char in input_text:                             # char(character) 변수를 input_text의 처음부터 끝까지
        if char not in input_characters:                # 만약 char가 input_characters(집합)에 없다면  
            input_characters.add(char)                  # 집합은 문자열을 문자단위로 나누어 원소로 가지고, 중복을 제거한다. 따라서 문장이였던 char에서 중복을 제거한 문자들이 input_characters(집합)에 추가된다.
    for char in target_text:                            # char(character) 변수를 input_text의 처음부터 끝까지
        if char not in target_characters:               # 만약 char가 input_characters(집합)에 없다면  
            target_characters.add(char)                 # 집합에 char변수를 추가한다.


input_characters = sorted(list(input_characters))       # input_characters 집합(set)을 list로 형변환하여 정렬(sorted)하여 다시 input_characters로 저장한다, 내장함수 sorted : 오름차순 정렬
target_characters = sorted(list(target_characters))     # target_characters 집합(set)을 list로 형변환하여 정렬(sorted)하여 다시 target_characters로 저장한다.
num_encoder_tokens = len(input_characters)              # input_characters(인풋문장의 문자(char)집합)의 개수를 num_encoder_tokens에 저장
num_decoder_tokens = len(target_characters)             # target_characters(타겟문장의 문자(char)집합)의 개수를 num_decoder_tokens에 저장
max_encoder_seq_length = max([len(txt) for txt in input_texts])     # input_texts 리스트 목록의 각각의 항목 txt에 대하여 len(txt)를 리스트로 저장하고 그 중 최대값을 찾는다.
max_decoder_seq_length = max([len(txt) for txt in target_texts])    # target_texts 리스트 목록의 각각의 항목 txt에 대하여 len(txt)를 리스트로 저장하고 그 중 최대값을 찾는다.

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


# 문지 -> 숫자 변환용 딕셔너리
input_token_index = dict(                                           #input_token_index에 딕셔너리를 생성
    [(char, i) for i, char in enumerate(input_characters)])         # enumerate는 반복문 사용 시 몇 번째 반복문인지 확인이 필요할 때 사용, 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환
                                                                    # 결과 : {' ': 0, '!': 1, '"': 2, '#': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, '*': 9, '+': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20,....}
                                                                    # 인덱스가 0~719까지 붙음. 이는 num_encoder_tokens가 720개로 출력되는 것과 같다는 것을 확인할 수 있다.
target_token_index = dict(                          
    [(char, i) for i, char in enumerate(target_characters)])        # enumerate 사용시 결과가 (a,b)라면 b에 컬렉션의 원소, a에 인덱스가 드어가기 때문에 for i, char로 받아와서 (char, i)로 저장


# 학습에 사용할 데이터를 담을 3차원 배열 : 문장들을 3차원 배열(encoder_input_data, decoder_input_data, decoder_target_data)로 변환합니다
encoder_input_data = np.zeros(                                      # np.zeros()는 인수로 들어온 배열의 크기만큼 0모든 원소가 인 배열을 생성한다.
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), # len(input_texts) = 2503 , max_encoder_seq_length = 73, num_encoder_tokens = 720, 따라서 크기가 (2503, 73, 720)이고
    dtype='float32')                                                # 데이터 타입은 실수형이며 모든 원소가 0인 3차원 배열을 생성한다, 인코더 인풋 데이터 = (2503, 73, 720)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), # 디코더 인풋 데이터 = (2503, 79, 688)
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), # 디코더 타겟 데이터 = (2503, 79, 688)
    dtype='float32')


# 문장들을 3차원 배열(encoder_input_data, decoder_input_data, decoder_target_data)로 변환합니다.
# encoder_input_data는 (num_pairs, max_english_sentence_length, num_english_characters) 형태의 3차원 배열로 영어 문장의 one-hot 형식 벡터 데이터를 갖고 있습니다.
# decoder_input_data는 (num_pairs, max_french_sentence_length, num_french_characters)형태의 3차원 배열로 불어 문장의 one-hot형식 벡터 데이터를 갖고 있습니다
# 이 과정이 seq2seq의 핵심인거 같으나 이해가 잘 안됌. 컨텍스트 벡터를 만들 때 원핫 임베딩이 아니라 그냥 워드 임베딩으로 처리하면 어떨까? 나중에 시도해보기.

# 문장을 문자 단위로 원 핫 인코딩하면서 학습용 데이터 생성                      # 내장함수 zip()은 같은 길이의 리스트를 같은 인덱스끼리 잘라서 리스트로 반환
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):  # 길이가 같은 input_texts와 target_texts를 묶어 하나의 리스트의 원소(input_text, target_text)로 만들고 enumerate로 인덱스(i)를 붙인다.
    for t, char in enumerate(input_text):                                       # input_text의 원소(char)와 인덱스(t)를 반복하여 생성한다
        encoder_input_data[i, t, input_token_index[char]] = 1.                  # 크기가(len(input_texts), max_encoder_seq_length, num_encoder_tokens)이고 모든 원소가 0인 3차원 배열 encoder_input_data에 [i, t, char를 키 값으로 가지는 딕셔너리의 인덱스] = 1. 으로 변경한다.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.                  # [i, t+1부터 끝까지, 나머지 모든 키값에 대한 인덱스]를 1.로 인코딩
    for t, char in enumerate(target_text):                                      # target_text의 원소(char)와 인덱스(t)를 target_text의 끝까지 반복 :
        # decoder_target_data is ahead of decoder_input_data by one timestep    # decoder_target_data는 decoder_input_data와 같지만 하나의 time step만큼 offset 됩니다.
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:                                                               # t가 0보다 크다면 :
            # decoder_target_data will be ahead by one timestep                 
            # and will not include the start character.                         
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.        # decoder_target_data[:, t, :]는 decoder_input_data[:, t + 1, :]와 같습니다.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.                 #
    decoder_target_data[i, t:, target_token_index[' ']] = 1.                    #



# 인코더 생성, 입력 시퀀스의 정의와 처리
encoder_inputs = Input(shape=(None, num_encoder_tokens))        # shape=(None, 720) 정수형 또는 None으로 구성된 형태 튜플(shape tuple)의 input_shape 인자를 첫번째 레이어에 전달합니다. 
                                                                # 여기서 None은 음이 아닌 어떠한 정수를 받을 수 있음을 의미합니다. 참고로 input_shape에는 배치 차원은(batch dimension) 포함되지 않습니다.
encoder = LSTM(latent_dim, return_state=True)                   # LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)     # encoder_inputs(NONE, 720)을 LSTM에 넣어 상태만 가져온다. 인코터 인풋의 각 시점대해서 입력과 은닉 상태를 가지고 연산
# `encoder_outputs`는 버리고 상태(`state_h, state_c`)는 유지
encoder_states = [state_h, state_c]                             


# `encoder_states`를 초기 상태로 사용해 decoder를 설정
decoder_inputs = Input(shape=(None, num_decoder_tokens))        # shape=(NONE, )
# 전체 출력 시퀀스를 반환하고 내부 상태도 반환하도록 decoder를 설정. 
# 학습 모델에서 상태를 반환하도록 하진 않지만, inference에서 사용할 예정.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)       # return_sequences=True를 선택하면 메모리 셀이 모든 시점(time step)에 대해서 은닉 상태값을 출력(False는 마지막 time stop에서의 메모리 셀의 은닉 상태값만 출력)
                                                                # 마지막 은닉 상태만 전달하도록 하면 many-to-one 문제를 풀 수 있고, 모든 시점의 은닉 상태를 전달하도록 하면, 다음층에 은닉층이 하나 더 있는 경우이거나 many-to-many 문제를 풀 수 있습니다.
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,            # dexoder_inputs과 초기상태를 인코더상태로 갖는 lstm을 사용하여 3D 텐서 반환
                                     initial_state=encoder_states)              
decoder_dense = Dense(num_decoder_tokens, activation='softmax') # 입력 은닉층과 출력 은닉층을 연결해주는 전결합층 Dense 사용. 출력 상태의 수를 num_decoder_tokens로 설정하고 다중 클래스 분류 문제에 주로 사용하는 softmax 활성화 함수 사용
                                                                # softmax는 입력 뉴런과 가중치를 계산한 값을 각 클래스의 확률 개념으로 표현할 수 있다. sigmoid = 0~1(이진분류), Relu=은닉층에 주로 사용, linear=입력 뉴런과 가중치 계산값 그대로 출력
decoder_outputs = decoder_dense(decoder_outputs)                # LSTM을 통해 계산된 decoder_outputs을 dense 레이어로 결합시켜 다시 decoder_outputs으로 저장



# Define the model that will turn 모델 정의
# `encoder_input_data`와 `decoder_input_data`를 `decoder_target_data`로 반환하도록 모델을 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)         # 앞서 정의해놓은 형식에 의한 모델 생성

# model.summary()
'''
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None, 720)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 688)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 1024), (None 7147520     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 1024), 7016448     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
                                                                 lstm_1[0][2]                     
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, None, 688)    705200      lstm_2[0][0]                     
==================================================================================================
Total params: 14,869,168
Trainable params: 14,869,168
Non-trainable params: 0
__________________________________________________________________________________________________
'''

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',             # 모델 컴파일, 손실함수 : categorical_crossentropy(다중분류에 주로 사용 - 정답만큼의 클래스가 존재), metrics는 정확도로 측정, RMSprop : 일반적으로 RNN 옵티마이저로 많이 사용
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,        # 원 핫 인코딩한 벡터들을 이용하고, [인코더인풋,디코더인풋]을 통해 디코터 타겟 데이터를 얻어낸다. 배치 사이즈와 에포크, 검증 데이터 비율은 앞서 지정한 것과 같다.
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')                                                            # 훈련이 끝나고 모델 저장. 추후에 불러와서 재사용할 수 있다.



# 다음: 추론 모드 (샘플링).
# 1) 입력문장을 encode하고 초기 상태에 decoder의 상태를 가지고 옵니다.
# 2) 초기 상태 decoder의 한 단계와 “시퀀스 시작” 토큰을 목표로 실행합니다. 출력은 다음 목표 문자입니다.
# 3) 예측된 목표 문자를 붙이고 이를 반복합니다. 다음은 추론을 설정한 부분입니다.

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
# encoder_model.summary()
'''
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_7 (InputLayer)         (None, None, 720)         0         
_________________________________________________________________
lstm_7 (LSTM)                [(None, 1024), (None, 102 7147520   
=================================================================
Total params: 7,147,520
Trainable params: 7,147,520
Non-trainable params: 0
_________________________________________________________________
'''

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
# decoder_model.summary()
'''
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_10 (InputLayer)           (None, None, 688)    0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 1024)         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 1024)         0                                            
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, None, 1024), 7016448     input_10[0][0]                   
                                                                 input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, None, 688)    705200      lstm_10[1][0]                    
==================================================================================================
Total params: 7,721,648
Trainable params: 7,721,648
Non-trainable params: 0
__________________________________________________________________________________________________
'''


# 역방향 조회(Reverse_lookup) 토큰 인덱스로 시퀀스를 다시 읽을 수 있는 것으로 디코딩.
# input_token_index_items()=dict_items : ([(' ', 0), ('!', 1), ('"', 2), ('#', 3), ('%', 4), ('&', 5), ("'", 6), ('(', 7), (')', 8), ('*', 9), ('+', 10), (',', 11), ('-', 12), ('.', 13), 
# ('/', 14), ('0', 15), ('1', 16), ('2', 17), ('3', 18), ('4', 19), ('5', 20), ('6', 21), ('7', 22), ('8', 23), ('9', 24), (':', 25)......])
# 이러한 아이템의 문자를 char로, 인덱스를 i로 받아온다.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # 상태 벡터로서 입력값을 encode
    states_value = encoder_model.predict(input_seq)

    # 길이가 1인 빈 목표 시퀀스를 생성
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 대상 시퀀스 첫 번째 문자를 시작 문자로 기재.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 시퀀스들의 batch에 대한 샘플링 반복(간소화를 위해, 배치 크기는 1로 상정)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 토큰으로 샘플링
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 탈출 조건 : 최대 길이에 도달하거나
        # 종료 문자를 찾을 경우
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # (길이 1인) 목표 시퀀스 최신화
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태 최신화
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Using TensorFlow backend.


Number of samples: 5723
Number of unique input tokens: 784
Number of unique output tokens: 701
Max sequence length for inputs: 73
Max sequence length for outputs: 79
Train on 4578 samples, validate on 1145 samples
Epoch 1/40
4578/4578 [==============================] - 30s 7ms/step - loss: 1.0686 - accuracy: 0.8406 - val_loss: 0.5731 - val_accuracy: 0.9071
Epoch 2/40
4578/4578 [==============================] - 28s 6ms/step - loss: 0.8222 - accuracy: 0.8593 - val_loss: 0.5174 - val_accuracy: 0.9070
Epoch 3/40
4578/4578 [==============================] - 29s 6ms/step - loss: 0.7938 - accuracy: 0.8621 - val_loss: 0.4256 - val_accuracy: 0.9102
Epoch 4/40
4578/4578 [==============================] - 29s 6ms/step - loss: 0.6828 - accuracy: 0.8724 - val_loss: 0.3246 - val_accuracy: 0.9273
Epoch 5/40
4578/4578 [==============================] - 28s 6ms/step - loss: 0.5767 - accuracy: 0.8898 - val_loss: 0.2550 - val_accuracy: 0.9489
Epoch 6/40
3392/4578 [=====================>........] - ETA: 

# 디버깅

In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

# epochs와 latent_dim의 초기값은 100, 256입니다. 하지만 낮을수록 훈련에 실패할 확률이 있으니 높게 설정해보세요
batch_size = 64  # Batch size for training. 한번에 훈련시킬 배치사이즈 설정
epochs = 200 # Number of epochs to train for. # 초기 한-영 번역기 훈련시 에포크 1회에 5분, 총 500분 이상 소요되었습니다. 코랩에서 GPU설정을 해주세요.
#또한 epochs 45번째 이상에서 약간의 과대적합이 일어나고 더이상 성능이 향상되지 않았습니다. / 2020-04-28 latent_dim을 1024로 설정했을 때 epochs 36 이상이면 과대적합 발생
# GPU를 사용해주세요. 
latent_dim = 1024  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on. 학습데이터 개수
# Path to the data txt file on disk.
data_path = '/content/drive/My Drive/캐시카우_노하람인턴_공유폴더/seq2seq/pre-post_TRIM.txt' #제작한 데이터 경로를 설정해주세요, 텍스트 파일이 UTF-8로 인코딩 되어있지 않으면 오류가 발생합니다.


# Vectorize the data.
input_texts = [] # 빈 리스트 생성
target_texts = [] # 빈 리스트 생성
input_characters = set() # 빈 집합 생성
target_characters = set() # 빈 집합 생성
# 중복을 허용하지 않는 set의 특징은 자료형의 중복을 제거하기 위한 필터 역할로 종종 사용하기도 한다.


with open(data_path, 'r', encoding='utf-8') as f:       # data_path를 읽기모드로, utf-8로 인코딩하여 f라는 이름으로 로드한다
    lines = f.read().split('\n')                        # f를 \n을 기준으로 나누어 한 줄씩 읽어 lines 변수에 저장한다.
for line in lines[: min(num_samples, len(lines) - 1)]:  # line을 lines까지, 처음부터 lines의 개수
                                                        # 훈련 샘플의 개수(10000)개와 실제 로드한 데이터의 개수(len(lines)-1)을 비교하여 최고값을 리턴하고, 처음부터 최소값까지 반복한다.
                                                        # 내장함수 min() : 반복가능한 객체의 가장 작은 요소 값을 리턴
    input_text, target_text, _ = line.split('\t')       # input_text(인풋), target_text(아웃풋), _(인덱스)는 한 문장으로 구성되어 있어 각각의 요소를 \t(TAB)으로 구분한다.
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.    "\t"문자를 시작 문자로, "\n"문자를 종료 문자로 사용.
    target_text = '\t' + target_text + '\n'             # targer_text는 앞에 '\t', 뒤에 '\n'을 붙인다.
    input_texts.append(input_text)                      # input_texts 라는 빈 리스트에 input_text를 하나씩 집어넣는다. 
    target_texts.append(target_text)                    # target_texts 라는 빈 리스트에 target_text를 하나씩 집어넣는다. 
    for char in input_text:                             # char(character) 변수를 input_text의 처음부터 끝까지
        if char not in input_characters:                # 만약 char가 input_characters(집합)에 없다면  
            input_characters.add(char)                  # 집합은 문자열을 문자단위로 나누어 원소로 가지고, 중복을 제거한다. 따라서 문장이였던 char에서 중복을 제거한 문자들이 input_characters(집합)에 추가된다.
    for char in target_text:                            # char(character) 변수를 input_text의 처음부터 끝까지
        if char not in target_characters:               # 만약 char가 input_characters(집합)에 없다면  
            target_characters.add(char)                 # 집합에 char변수를 추가한다.


input_characters = sorted(list(input_characters))       # input_characters 집합(set)을 list로 형변환하여 정렬(sorted)하여 다시 input_characters로 저장한다, 내장함수 sorted : 오름차순 정렬
target_characters = sorted(list(target_characters))     # target_characters 집합(set)을 list로 형변환하여 정렬(sorted)하여 다시 target_characters로 저장한다.
num_encoder_tokens = len(input_characters)              # input_characters(인풋문장의 문자(char)집합)의 개수를 num_encoder_tokens에 저장
num_decoder_tokens = len(target_characters)             # target_characters(타겟문장의 문자(char)집합)의 개수를 num_decoder_tokens에 저장
max_encoder_seq_length = max([len(txt) for txt in input_texts])     # input_texts 리스트 목록의 각각의 항목 txt에 대하여 len(txt)를 리스트로 저장하고 그 중 최대값을 찾는다.
max_decoder_seq_length = max([len(txt) for txt in target_texts])    # target_texts 리스트 목록의 각각의 항목 txt에 대하여 len(txt)를 리스트로 저장하고 그 중 최대값을 찾는다.

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


# 문지 -> 숫자 변환용 딕셔너리
input_token_index = dict(                                           #input_token_index에 딕셔너리를 생성
    [(char, i) for i, char in enumerate(input_characters)])         # enumerate는 반복문 사용 시 몇 번째 반복문인지 확인이 필요할 때 사용, 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환
                                                                    # 결과 : {' ': 0, '!': 1, '"': 2, '#': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, '*': 9, '+': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20,....}
                                                                    # 인덱스가 0~719까지 붙음. 이는 num_encoder_tokens가 720개로 출력되는 것과 같다는 것을 확인할 수 있다.
target_token_index = dict(                          
    [(char, i) for i, char in enumerate(target_characters)])        # enumerate 사용시 결과가 (a,b)라면 b에 컬렉션의 원소, a에 인덱스가 드어가기 때문에 for i, char로 받아와서 (char, i)로 저장


# 학습에 사용할 데이터를 담을 3차원 배열 : 문장들을 3차원 배열(encoder_input_data, decoder_input_data, decoder_target_data)로 변환합니다
encoder_input_data = np.zeros(                                      # np.zeros()는 인수로 들어온 배열의 크기만큼 0모든 원소가 인 배열을 생성한다.
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), # len(input_texts) = 2503 , max_encoder_seq_length = 73, num_encoder_tokens = 720, 따라서 크기가 (2503, 73, 720)이고
    dtype='float32')                                                # 데이터 타입은 실수형이며 모든 원소가 0인 3차원 배열을 생성한다, 인코더 인풋 데이터 = (2503, 73, 720)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), # 디코더 인풋 데이터 = (2503, 79, 688)
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), # 디코더 타겟 데이터 = (2503, 79, 688)
    dtype='float32')


# 문장들을 3차원 배열(encoder_input_data, decoder_input_data, decoder_target_data)로 변환합니다.
# encoder_input_data는 (num_pairs, max_english_sentence_length, num_english_characters) 형태의 3차원 배열로 영어 문장의 one-hot 형식 벡터 데이터를 갖고 있습니다.
# decoder_input_data는 (num_pairs, max_french_sentence_length, num_french_characters)형태의 3차원 배열로 불어 문장의 one-hot형식 벡터 데이터를 갖고 있습니다
# 이 과정이 seq2seq의 핵심인거 같으나 이해가 잘 안됌. 컨텍스트 벡터를 만들 때 원핫 임베딩이 아니라 그냥 워드 임베딩으로 처리하면 어떨까? 나중에 시도해보기.

# 문장을 문자 단위로 원 핫 인코딩하면서 학습용 데이터 생성                      # 내장함수 zip()은 같은 길이의 리스트를 같은 인덱스끼리 잘라서 리스트로 반환
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):  # 길이가 같은 input_texts와 target_texts를 묶어 하나의 리스트의 원소(input_text, target_text)로 만들고 enumerate로 인덱스(i)를 붙인다.
    for t, char in enumerate(input_text):                                       # input_text의 원소(char)와 인덱스(t)를 반복하여 생성한다
        encoder_input_data[i, t, input_token_index[char]] = 1.                  # 크기가(len(input_texts), max_encoder_seq_length, num_encoder_tokens)이고 모든 원소가 0인 3차원 배열 encoder_input_data에 [i, t, char를 키 값으로 가지는 딕셔너리의 인덱스] = 1. 으로 변경한다.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.                  # [i, t+1부터 끝까지, 나머지 모든 키값에 대한 인덱스]를 1.로 인코딩
    for t, char in enumerate(target_text):                                      # target_text의 원소(char)와 인덱스(t)를 target_text의 끝까지 반복 :
        # decoder_target_data is ahead of decoder_input_data by one timestep    # decoder_target_data는 decoder_input_data와 같지만 하나의 time step만큼 offset 됩니다.
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:                                                               # t가 0보다 크다면 :
            # decoder_target_data will be ahead by one timestep                 
            # and will not include the start character.                         
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.        # decoder_target_data[:, t, :]는 decoder_input_data[:, t + 1, :]와 같습니다.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.                 #
    decoder_target_data[i, t:, target_token_index[' ']] = 1.                    #



# 인코더 생성, 입력 시퀀스의 정의와 처리
encoder_inputs = Input(shape=(None, num_encoder_tokens))        # shape=(None, 720) 정수형 또는 None으로 구성된 형태 튜플(shape tuple)의 input_shape 인자를 첫번째 레이어에 전달합니다. 
                                                                # 여기서 None은 음이 아닌 어떠한 정수를 받을 수 있음을 의미합니다. 참고로 input_shape에는 배치 차원은(batch dimension) 포함되지 않습니다.
encoder = LSTM(latent_dim, return_state=True)                   # LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)     # encoder_inputs(NONE, 720)을 LSTM에 넣어 상태만 가져온다. 인코터 인풋의 각 시점대해서 입력과 은닉 상태를 가지고 연산
# `encoder_outputs`는 버리고 상태(`state_h, state_c`)는 유지
encoder_states = [state_h, state_c]                             


# `encoder_states`를 초기 상태로 사용해 decoder를 설정
decoder_inputs = Input(shape=(None, num_decoder_tokens))        # shape=(NONE, )
# 전체 출력 시퀀스를 반환하고 내부 상태도 반환하도록 decoder를 설정. 
# 학습 모델에서 상태를 반환하도록 하진 않지만, inference에서 사용할 예정.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)       # return_sequences=True를 선택하면 메모리 셀이 모든 시점(time step)에 대해서 은닉 상태값을 출력(False는 마지막 time stop에서의 메모리 셀의 은닉 상태값만 출력)
                                                                # 마지막 은닉 상태만 전달하도록 하면 many-to-one 문제를 풀 수 있고, 모든 시점의 은닉 상태를 전달하도록 하면, 다음층에 은닉층이 하나 더 있는 경우이거나 many-to-many 문제를 풀 수 있습니다.
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,            # dexoder_inputs과 초기상태를 인코더상태로 갖는 lstm을 사용하여 3D 텐서 반환
                                     initial_state=encoder_states)              
decoder_dense = Dense(num_decoder_tokens, activation='softmax') # 입력 은닉층과 출력 은닉층을 연결해주는 전결합층 Dense 사용. 출력 상태의 수를 num_decoder_tokens로 설정하고 다중 클래스 분류 문제에 주로 사용하는 softmax 활성화 함수 사용
                                                                # softmax는 입력 뉴런과 가중치를 계산한 값을 각 클래스의 확률 개념으로 표현할 수 있다. sigmoid = 0~1(이진분류), Relu=은닉층에 주로 사용, linear=입력 뉴런과 가중치 계산값 그대로 출력
decoder_outputs = decoder_dense(decoder_outputs)                # LSTM을 통해 계산된 decoder_outputs을 dense 레이어로 결합시켜 다시 decoder_outputs으로 저장



# Define the model that will turn 모델 정의
# `encoder_input_data`와 `decoder_input_data`를 `decoder_target_data`로 반환하도록 모델을 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)         # 앞서 정의해놓은 형식에 의한 모델 생성

# model.summary()
'''
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, None, 720)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 688)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 1024), (None 7147520     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 1024), 7016448     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
                                                                 lstm_1[0][2]                     
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, None, 688)    705200      lstm_2[0][0]                     
==================================================================================================
Total params: 14,869,168
Trainable params: 14,869,168
Non-trainable params: 0
__________________________________________________________________________________________________
'''
'''
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',             # 모델 컴파일, 손실함수 : categorical_crossentropy(다중분류에 주로 사용 - 정답만큼의 클래스가 존재), metrics는 정확도로 측정, RMSprop : 일반적으로 RNN 옵티마이저로 많이 사용
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,        # 원 핫 인코딩한 벡터들을 이용하고, [인코더인풋,디코더인풋]을 통해 디코터 타겟 데이터를 얻어낸다. 배치 사이즈와 에포크, 검증 데이터 비율은 앞서 지정한 것과 같다.
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')                                                            # 훈련이 끝나고 모델 저장. 추후에 불러와서 재사용할 수 있다.
'''


# 다음: 추론 모드 (샘플링).
# 1) 입력문장을 encode하고 초기 상태에 decoder의 상태를 가지고 옵니다.
# 2) 초기 상태 decoder의 한 단계와 “시퀀스 시작” 토큰을 목표로 실행합니다. 출력은 다음 목표 문자입니다.
# 3) 예측된 목표 문자를 붙이고 이를 반복합니다. 다음은 추론을 설정한 부분입니다.

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
# encoder_model.summary()
'''
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_7 (InputLayer)         (None, None, 720)         0         
_________________________________________________________________
lstm_7 (LSTM)                [(None, 1024), (None, 102 7147520   
=================================================================
Total params: 7,147,520
Trainable params: 7,147,520
Non-trainable params: 0
_________________________________________________________________
'''

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
# decoder_model.summary()
'''
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_10 (InputLayer)           (None, None, 688)    0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 1024)         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 1024)         0                                            
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, None, 1024), 7016448     input_10[0][0]                   
                                                                 input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, None, 688)    705200      lstm_10[1][0]                    
==================================================================================================
Total params: 7,721,648
Trainable params: 7,721,648
Non-trainable params: 0
__________________________________________________________________________________________________
'''


# 역방향 조회(Reverse_lookup) 토큰 인덱스로 시퀀스를 다시 읽을 수 있는 것으로 디코딩.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
print(input_token_index.items())

Using TensorFlow backend.


Number of samples: 2503
Number of unique input tokens: 720
Number of unique output tokens: 688
Max sequence length for inputs: 73
Max sequence length for outputs: 79
dict_items([(' ', 0), ('!', 1), ('"', 2), ('#', 3), ('%', 4), ('&', 5), ("'", 6), ('(', 7), (')', 8), ('*', 9), ('+', 10), (',', 11), ('-', 12), ('.', 13), ('/', 14), ('0', 15), ('1', 16), ('2', 17), ('3', 18), ('4', 19), ('5', 20), ('6', 21), ('7', 22), ('8', 23), ('9', 24), (':', 25), (';', 26), ('<', 27), ('=', 28), ('>', 29), ('?', 30), ('A', 31), ('B', 32), ('C', 33), ('D', 34), ('E', 35), ('F', 36), ('G', 37), ('H', 38), ('I', 39), ('J', 40), ('K', 41), ('L', 42), ('M', 43), ('N', 44), ('O', 45), ('P', 46), ('Q', 47), ('R', 48), ('S', 49), ('T', 50), ('U', 51), ('V', 52), ('W', 53), ('X', 54), ('Y', 55), ('Z', 56), ('[', 57), (']', 58), ('_', 59), ('a', 60), ('b', 61), ('c', 62), ('d', 63), ('e', 64), ('f', 65), ('g', 66), ('h', 67), ('i', 68), ('j', 69), ('k', 70), ('l', 71), ('m', 72), ('n', 73), ('o', 74), ('p', 7